In [1]:
import pandas as pd
import numpy as np
import os
import random

import warnings

warnings.filterwarnings(action='ignore')

k = 100 # k개까지만 문제 기록을 남기려고함.

path = '/opt/ml/input/data/'

train = pd.read_csv(path + 'train_data.csv')
test = pd.read_csv(path + 'test_data.csv')
dat = pd.concat([train, test])
dat = dat.loc[::-1]
dat.reset_index(drop= True, inplace= True)

dat['tem'] = 1
dat['seq'] = dat.groupby('userID')['tem'].apply(lambda x : x.cumsum())

In [2]:
# test 데이터 만들기.
test_id = test['userID'].unique()
test = dat[(dat['userID'].isin(test_id)) & (dat['seq'] <= k)]
train = dat[~((dat['userID'].isin(test_id)) & (dat['seq'] == 1))]
train['seq'] = train.apply(lambda x : x['seq'] - 1  if x['userID'] in test_id else x['seq'], axis = 1)

# valid 데이터 만들기.
valid_data = train[train['seq'] <= k]

# 뒷처리부분.
user_num = dat['userID'].nunique()
train_new = train.copy()

# seq 하나씩 내리고 0된 값 처리하기.
train_new['seq'] = train_new['seq'] - 1
train_new = train_new[train_new['seq'] >= 1]

# train_new 데이터에 문제 너무 적게 푼 유저 정리하기.
tem = train_new.drop_duplicates(['userID'], keep='last')[['userID','seq']]
lst = list(tem[tem['seq'] < 14]['userID'])
train_new = train_new[~train_new['userID'].isin(lst)]

# 첫번째 train_datas 만들기.
train_datas = train_new[train_new['seq'] <= k]

In [3]:
for i in range(300):
    # seq 하나씩 내리고 0된 값 처리하기.
    train_new['seq'] = train_new['seq'] - 1
    train_new = train_new[train_new['seq'] >= 1]
    
    # 문제를 적게 푼 유저 정리하기.
    tem = train_new.drop_duplicates(['userID'], keep='last')[['userID','seq']]
    lst = list(tem[tem['seq'] < 14]['userID'])
    train_new = train_new[~train_new['userID'].isin(lst)]
    train_new['userID'] = train_new['userID'] + user_num
    
    # train_data 만들기.
    train_data = train_new[train_new['seq'] <= k]
    train_datas = pd.concat([train_datas, train_data])

    if i % 50 == 0:
        print(i)

0
50
100
150
200
250


In [7]:
train_datas['userID'].value_counts()

885826     100
1199933    100
1199924    100
1199926    100
1199928    100
          ... 
2013429     14
2013790     14
718396      14
354917      14
7439        14
Name: userID, Length: 1386576, dtype: int64

In [9]:
train_data = train_datas.sort_values(['userID','seq'], ascending=[True, False])
valid_data = valid_data.sort_values(['userID','seq'], ascending=[True, False])
test_data = test.sort_values(['userID','seq'], ascending=[True, False])

In [12]:
train_data['userID'].nunique()

1386576

In [13]:
valid_data['userID'].nunique()

7442

In [15]:
train_data.drop(['tem'], axis = 1).to_csv(path + 'GKT/train_data3.csv', index = False)
valid_data.drop(['tem'], axis = 1).to_csv(path + 'GKT/valid_data3.csv', index = False)
test_data.drop(['tem'], axis = 1).to_csv(path + 'GKT/test_data3.csv', index = False)

In [ ]:
train_data.drop(['tem'], axis = 1).to_csv('./train_data.csv', index = False)
valid_data.drop(['tem'], axis = 1).to_csv('./valid_data.csv', index = False)
test_data.drop(['tem'], axis = 1).to_csv('./test_data.csv', index = False)